In [1]:
# Importing the required libraries
import os
import random
import pandas as pd

import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
from ultralytics import YOLO

In [2]:
# Define the paths to the images and labels directories
train_images = 'C:/Users/Shahid Ahamed Hasib/Downloads/archive/train/images'
train_labels = 'C:/Users/Shahid Ahamed Hasib/Downloads/archive/train/labels'

val_images = 'C:/Users/Shahid Ahamed Hasib/Downloads/archive/valid/images'
val_labels = 'C:/Users/Shahid Ahamed Hasib/Downloads/archive/valid/labels'

test_images = 'C:/Users/Shahid Ahamed Hasib/Downloads/archive/test/images'
test_labels = 'C:/Users/Shahid Ahamed Hasib/Downloads/archive/test/labels'

In [3]:
# Define the path to the yaml data file
yaml_path = 'C:/Users/Shahid Ahamed Hasib/Downloads/archive/data.yaml'

In [4]:
# Read a image by path
image_path = os.path.join(train_images, os.listdir(train_images)[100])
image = cv2.imread(image_path)

# Get the size of the image
height, width, channels = image.shape
print('The image has dimensions {}x{} and {} channels'.format(height, width, channels))

The image has dimensions 1024x768 and 3 channels


In [5]:
# Define the labels
classes = ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']
Idx2Label = {idx: label for idx, label in enumerate(classes)}
Label2Index = {label: idx for idx, label in Idx2Label.items()}
print('Index to Label Mapping:', Idx2Label)
print('Label to Index Mapping:', Label2Index)

Index to Label Mapping: {0: 'fish', 1: 'jellyfish', 2: 'penguin', 3: 'puffin', 4: 'shark', 5: 'starfish', 6: 'stingray'}
Label to Index Mapping: {'fish': 0, 'jellyfish': 1, 'penguin': 2, 'puffin': 3, 'shark': 4, 'starfish': 5, 'stingray': 6}


In [6]:
def visualize_image_with_annotation_bboxes(image_dir, label_dir):
    # Get list of all the image files in the directory
    image_files = sorted(os.listdir(image_dir))
    
    # Choose 10 random image files from the list
    sample_image_files = random.sample(image_files, 12)
    
    # Set up the plot
    fig, axs = plt.subplots(4, 3, figsize=(15, 20))
    
    # Loop over the random images and plot the bounding boxes
    for i, image_file in enumerate(sample_image_files):
        row = i // 3
        col = i % 3
        
        # Load the image
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load the labels for this image
        label_path = os.path.join(label_dir, image_file[:-4] + '.txt')
        f = open(label_path, 'r')
        
        # Loop over the labels and plot the bounding boxes
        for label in f:
            class_id, x_center, y_center, width, height = map(float, label.split())
            h, w, _ = image.shape
            x_min = int((x_center - width/2) * w)
            y_min = int((y_center - height/2) * h)
            x_max = int((x_center + width/2) * w)
            y_max = int((y_center + height/2) * h)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(image, Idx2Label[int(class_id)], (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    
        axs[row, col].imshow(image)
        axs[row, col].axis('off')

    #plt.show()
    plt.savefig('output_image_grid.png')  # Save the figure as an image file
    plt.close()  # Close the plot to avoid placeholder output
    print("Figure saved as 'output_image_grid.png'")

# Visualize 12 sample images with bounding boxes
visualize_image_with_annotation_bboxes(train_images, train_labels)

Figure saved as 'output_image_grid.png'


In [11]:
#torch.cuda.is_available()
#torch.cuda.empty_cache()

In [14]:
# Load a YOLOv8 model, for example, the nano version for fast training
model = YOLO('yolov8n.pt')  # You can use 'yolov8s.pt', 'yolov8m.pt', etc. for different sizes
# Train the model
results = model.train(
    data= yaml_path ,  # Path to the dataset YAML file
    epochs=1,                                 # Number of training epochs (adjust as needed)
    imgsz=(height, width, channels),                                 # Image size for training (can be adjusted)
    lr0=0.1,                                  # Initial learning rate
    lrf=0.01,                                   # Final learning rate (fraction of initial LR)
    seed=42,
    batch=8,                                   # Batch size (adjust based on your CPU/GPU memory)
    workers=4,
    device='gpu',                              # Using 'cpu' since CUDA is not available
    name='underwater_detection'                # Folder name for saving the model and results
)

# Extract the mAP score
maP_50 = results.metrics['metrics/mAP50(B)']   # mAP@0.5 score
print(f"mAP@0.5: {maP_50:.4f}")

New https://pypi.org/project/ultralytics/8.3.11 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.10  Python-3.12.3 torch-2.4.1+cpu 


ValueError: Invalid CUDA 'device=gpu' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: 0
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
%matplotlib inline
# read in the results.csv file as a pandas dataframe
df = pd.read_csv('path.csv')
df.columns = df.columns.str.strip()

# create subplots using seaborn
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))

# plot the columns using seaborn
sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0,0])
sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0,1])
sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1,0])
sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1,1])
sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2,0])
sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2,1])
sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3,0])
sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3,1])
sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4,0])
sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4,1])

# set titles and axis labels for each subplot
axs[0,0].set(title='Train Box Loss')
axs[0,1].set(title='Train Class Loss')
axs[1,0].set(title='Train DFL Loss')
axs[1,1].set(title='Metrics Precision (B)')
axs[2,0].set(title='Metrics Recall (B)')
axs[2,1].set(title='Metrics mAP50 (B)')
axs[3,0].set(title='Metrics mAP50-95 (B)')
axs[3,1].set(title='Validation Box Loss')
axs[4,0].set(title='Validation Class Loss')
axs[4,1].set(title='Validation DFL Loss')

# add suptitle and subheader
plt.suptitle('Training Metrics and Loss', fontsize=24)

# adjust top margin to make space for suptitle
plt.subplots_adjust(top=0.8)

# adjust spacing between subplots
plt.tight_layout()

plt.show()

In [ ]:
# Loading the best performing model
model = YOLO('path/runs/detect/yolov8n_custom/weights/best.pt')

# Evaluating the model on test dataset
metrics = model.val(conf=0.25, split='test')

In [ ]:
# Function to perform detections with trained model
def predict_detection(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Pass the image through the detection model and get the result
    detect_result = model(image)
    
    # Plot the detections
    detect_image = detect_result[0].plot()
    
    # Convert the image to RGB format
    detect_image = cv2.cvtColor(detect_image, cv2.COLOR_BGR2RGB)
    
    return detect_image

In [ ]:
# Get list of all the image files in the test directory
image_files = sorted(os.listdir(test_images))
    
# Choose 12 random image files from the list
sample_image_files = random.sample(image_files, 12)
    
# Set up the plot
fig, axs = plt.subplots(4, 3, figsize=(15, 20))
    
# Loop over the random images and plot the detections of the trained model
for i, image_file in enumerate(sample_image_files):
    row = i // 3
    col = i % 3
        
    # Load the current image and run object detection
    image_path = os.path.join(test_images, image_file)
    detect_image = predict_detection(image_path)
    
    axs[row, col].imshow(detect_image)
    axs[row, col].axis('off')

plt.show()